In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from sklearn.metrics import accuracy_score
from torchsummary import summary

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.485, 0.485],
        std=[0.229, 0.229, 0.229]
    )

])

In [4]:
trainData=torchvision.datasets.ImageFolder(
    root='/content/drive/MyDrive/Data/BrainTumorDataset/Training',
    transform=transform,
)
testData=torchvision.datasets.ImageFolder(
    root='/content/drive/MyDrive/Data/BrainTumorDataset/Testing1',
    transform=transform,
)

In [17]:
testData2=torchvision.datasets.ImageFolder(
    root='/content/drive/MyDrive/Data/BrainTumorDataset/Testing',
    transform=transform,
)

In [6]:
loadedTrainData=DataLoader(trainData,batch_size=32,shuffle=True,num_workers=4)
loadedTestData=DataLoader(testData,batch_size=32,shuffle=False,num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [18]:
loadedTestData2=DataLoader(testData2,batch_size=32,shuffle=False,num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [5]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, se_ratio=0.25):
        super().__init__()
        reduced_channels = max(1, int(in_channels * se_ratio))
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, reduced_channels, kernel_size=1),
            nn.SiLU(),
            nn.Conv2d(reduced_channels, in_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

In [7]:
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, expansion=1, kernel_size=3, stride=1, se_ratio=0.25):
        super().__init__()
        expanded_channels = in_channels * expansion

        self.block = nn.Sequential()
        if expansion != 1:
            self.block.add_module("expand_conv", nn.Conv2d(in_channels, expanded_channels, kernel_size=1))
            self.block.add_module("expand_norm", nn.BatchNorm2d(expanded_channels))
            self.block.add_module("expand_act", nn.SiLU())

        self.block.add_module("depthwise_conv", nn.Conv2d(
            expanded_channels, expanded_channels, kernel_size=kernel_size,
            stride=stride, padding=kernel_size//2, groups=expanded_channels
        ))
        self.block.add_module("se", SqueezeExcitation(expanded_channels, se_ratio))
        self.block.add_module("project_conv", nn.Conv2d(expanded_channels, out_channels, kernel_size=1))
        self.block.add_module("project_norm", nn.BatchNorm2d(out_channels))

        self.use_residual = (stride == 1) and (in_channels == out_channels)
        self.dropout = nn.Dropout2d(0.2)

    def forward(self, x):
        if self.use_residual:
            return x + self.dropout(self.block(x))
        return self.block(x)

In [8]:
class BrainTumor(nn.Module):
  def __init__(self, inputNeurons,outputNeurons):
    super().__init__()
    self.stems=nn.Sequential(
        nn.Conv2d(inputNeurons,32,kernel_size=3,stride=2,padding=1),
        nn.BatchNorm2d(32),
        nn.SiLU(),
    )
    self.MbBlocks=nn.Sequential(
        MBConv(32,16,expansion=1,kernel_size=3,stride=1),
        MBConv(16,24,expansion=6,kernel_size=3,stride=2),
        MBConv(24,40,expansion=6,kernel_size=5,stride=2),
        MBConv(40,80,expansion=6,kernel_size=3,stride=2),
        MBConv(80,112,expansion=6,kernel_size=5,stride=1),
        MBConv(112,192,expansion=6,kernel_size=5,stride=2),
        MBConv(192,320,expansion=6,kernel_size=3,stride=1),
    )
    self.head=nn.Sequential(
        nn.Conv2d(320,1280,kernel_size=1),
        nn.BatchNorm2d(1280),
        nn.SiLU(),
        nn.AdaptiveAvgPool2d(1),
        nn.Dropout(0.2),
        nn.Flatten(),
        nn.Linear(1280,outputNeurons),
    )
  def forward(self,x):
      x=self.stems(x)
      x=self.MbBlocks(x)
      x=self.head(x)
      return x



In [9]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=BrainTumor(
    inputNeurons=3,
    outputNeurons=4,
).to(DEVICE)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)

In [10]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             896
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             320
 AdaptiveAvgPool2d-5             [-1, 32, 1, 1]               0
            Conv2d-6              [-1, 8, 1, 1]             264
              SiLU-7              [-1, 8, 1, 1]               0
            Conv2d-8             [-1, 32, 1, 1]             288
           Sigmoid-9             [-1, 32, 1, 1]               0
SqueezeExcitation-10         [-1, 32, 112, 112]               0
           Conv2d-11         [-1, 16, 112, 112]             528
      BatchNorm2d-12         [-1, 16, 112, 112]              32
           MBConv-13         [-1, 16, 112, 112]               0
           Conv2d-14         [-1, 96, 1

In [11]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: Tesla T4 is available.


In [12]:
epochs = 30
patience = 10  # Number of epochs to wait before stopping if no improvement
best_loss = float("inf")
counter = 0  # Count epochs with no improvement

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in loadedTrainData:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Compute training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_loss / len(loadedTrainData)
    train_accuracy = 100 * correct_train / total_train

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # Early Stopping Check
    if avg_train_loss < best_loss:
        best_loss = avg_train_loss
        counter = 0  # Reset patience counter
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}. No improvement in {patience} epochs.")
            break

# Final training loss and accuracy
print(f"Final Training Loss: {best_loss:.4f}, Final Training Accuracy: {train_accuracy:.2f}%")


Epoch 1/30, Loss: 0.7008, Accuracy: 73.13%
Epoch 2/30, Loss: 0.4724, Accuracy: 81.86%
Epoch 3/30, Loss: 0.3651, Accuracy: 86.54%
Epoch 4/30, Loss: 0.3123, Accuracy: 88.52%
Epoch 5/30, Loss: 0.2796, Accuracy: 89.97%
Epoch 6/30, Loss: 0.2415, Accuracy: 91.23%
Epoch 7/30, Loss: 0.1813, Accuracy: 93.47%
Epoch 8/30, Loss: 0.1941, Accuracy: 92.40%
Epoch 9/30, Loss: 0.1837, Accuracy: 93.40%
Epoch 10/30, Loss: 0.1619, Accuracy: 94.29%
Epoch 11/30, Loss: 0.1412, Accuracy: 94.70%
Epoch 12/30, Loss: 0.1401, Accuracy: 95.03%
Epoch 13/30, Loss: 0.1292, Accuracy: 95.06%
Epoch 14/30, Loss: 0.1285, Accuracy: 95.68%
Epoch 15/30, Loss: 0.1124, Accuracy: 95.85%
Epoch 16/30, Loss: 0.1145, Accuracy: 96.06%
Epoch 17/30, Loss: 0.1147, Accuracy: 96.18%
Epoch 18/30, Loss: 0.0987, Accuracy: 96.76%
Epoch 19/30, Loss: 0.0992, Accuracy: 96.45%
Epoch 20/30, Loss: 0.0998, Accuracy: 96.62%
Epoch 21/30, Loss: 0.0789, Accuracy: 97.20%
Epoch 22/30, Loss: 0.1084, Accuracy: 96.08%
Epoch 23/30, Loss: 0.0834, Accuracy: 97.1

In [13]:
model.eval()
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in loadedTestData:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute loss
        test_loss += loss.item()  # Accumulate loss

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
average_loss = test_loss / len(loadedTestData)

print(f"Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.2f}%")













Test Loss: 0.1324, Test Accuracy: 95.94%


In [20]:
#testData 2
model.eval()
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in loadedTestData2:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute loss
        test_loss += loss.item()  # Accumulate loss

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
average_loss = test_loss / len(loadedTestData2)

print(f"Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.2f}%")













/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Loss: 0.0843, Test Accuracy: 95.78%


In [15]:
scripted_model = torch.jit.script(model)
scripted_model.save("/content/drive/MyDrive/SavedModels/ptFiles/BrainTumorModel1.pt")

In [16]:
torch.save(model.state_dict(), "/content/drive/MyDrive/SavedModels/pthfiles/BrainTumorModel1.pth")
